In [9]:
from jax import numpy as jnp
from jax import random

from boax.prediction import kernels, means, models
from boax.util import const

In [20]:
key_1, key_2 = random.split(random.key(0))
index_points = random.uniform(key_1, shape=(10, 1), minval=-1, maxval=1)
observation_index_points = random.uniform(key_2, shape=(5, 1), minval=-1, maxval=1)
observations = jnp.sin(observation_index_points[..., 0]) + jnp.cos(observation_index_points[..., 0])

In [23]:
model = models.gaussian_process_regression(
  observation_index_points,
  observations,
  means.zero(),
  kernels.rbf(jnp.array(0.2)),
  jnp.array(0.2),
)

In [24]:
model(index_points)

(Array([-0.20518558,  0.00151967,  1.1631446 , -0.10359916,  0.13897833,
         1.1311573 ,  0.19742526,  0.00142203,  0.02079975,  0.02918658],      dtype=float32),
 Array([[ 1.71759889e-01,  1.71041873e-04,  1.75843845e-12,
          4.32922840e-02,  9.17909020e-08,  2.79247263e-12,
          3.44754092e-08,  2.14642918e-04,  4.35918264e-06,
          2.48269043e-06],
        [ 1.71041873e-04,  9.99993145e-01,  1.31821819e-03,
         -3.29662987e-04,  2.19622150e-01,  1.78865448e-03,
          1.56491965e-01,  9.97740984e-01,  6.44115627e-01,
          5.67718565e-01],
        [ 1.75843845e-12,  1.31821819e-03,  1.91351697e-01,
         -3.34483418e-12,  8.90367478e-02,  1.98617220e-01,
          1.14777476e-01,  1.05092407e-03,  1.90743934e-02,
          2.54455134e-02],
        [ 4.32922840e-02, -3.29662987e-04, -3.34483418e-12,
          1.03197262e-01, -1.74988628e-07, -5.31188071e-12,
         -6.56957937e-08, -4.14086942e-04, -8.33448121e-06,
         -4.74384024e-06],
    